In [1]:
from pyspark.sql import SparkSession
from urllib.request import urlretrieve
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ADS project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.executor.memory","4G")
    .config("spark.driver.memory","8G")
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/22 01:54:47 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.17.1.121 instead (on interface eth0)
22/08/22 01:54:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/22 01:54:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [4]:
YEARS = ['2019', '2021']
MONTHS = range(1, 13)
path = f"../data/curated/tlc_data/"
cols = ['is_weekend', 'is_airport', 'is_rainy']
count = 0


In [5]:
sdf = spark.read.parquet(f'{path}2021/final_data/*/')

In [6]:
# take the data of 2021 to aggregate with respect to the categorical features
aggregated_results = sdf \
                    .groupBy(cols) \
                    .agg(
                        # take the mean of each with respect to the combinations of categorical features
                        F.mean("tip_amount").alias("avg_tip_amount"),
                        F.mean("tip_rate").alias("avg_tip_rate")
                    ).orderBy('avg_tip_amount')
                    # order by Avg Tip Amount in ascending order
                    

aggregated_results.show()

+----------+----------+--------+------------------+-------------------+
|is_weekend|is_airport|is_rainy|    avg_tip_amount|       avg_tip_rate|
+----------+----------+--------+------------------+-------------------+
|         1|     false|   False|2.4369184440813187| 0.2727787806906374|
|         1|     false|    True| 2.439985836728571|0.27517731627348113|
|         0|     false|    True|2.4675789243910105|0.27978290572314785|
|         0|     false|   False| 2.474437247835323|0.27891717295346324|
|         1|      true|   False|3.1662441838823736|0.24214262662915156|
|         1|      true|    True| 3.193979561166214|0.24826461778725292|
|         0|      true|    True| 3.251899867374009| 0.2473157291784234|
|         0|      true|   False| 3.265285362931209|  0.245129772343385|
+----------+----------+--------+------------------+-------------------+

